In [40]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,AveragePooling2D,Dense,Flatten
import numpy as np 

In [ ]:
import os
import matplotlib as plt 
from matplotlib.pyplot import imread,imshow,get_cmap
train_dir="data/traindata/"
dir_list=os.listdir(train_dir)

img=imread(os.path.join(train_dir,(str(dir_list[0]))))
imshow(img)

In [ ]:
# now we have to classify the data manually.Means dogs and cats are kept in two sepertae
# folder.So that we can tell which belongs to which class.

import os,shutil
directory_path='data/traindata/'
sub_dir=os.listdir(directory_path)
# print(sub_dir)
file_path=os.path.join(directory_path,str(sub_dir[0]))

all_files=os.listdir(file_path)

cat_class=os.path.join(file_path,'cat')
dog_class=os.path.join(file_path,'dog')

# cat_dir=os.mkdir(cat_class)
# dog_dir=os.mkdir(dog_class)

for file in all_files:
    original=os.path.join(file_path,str(file))
    if str(file).split('.').count('cat')>0:
        target=os.path.join(file_path,'cat')
        shutil.move(original,target)
        print(str(file))
        #move to cat class 
    if str(file).split('.').count('dog')>0:
        target=os.path.join(file_path,'dog')
        shutil.move(original,target)
        print(str(file))
        #move to dog 
# print(all_files[0])

In [24]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
# https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/
train_data_gen=ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0.5,
        height_shift_range=0.5,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.20)

test_data_gen=ImageDataGenerator(
        rescale=1./255
)
# categorical means return a 2d hot encoded image

# one_hot_encoding : One-Hot Encoding is another popular technique for treating categorical variables. It simply creates additional features based on the number of unique values in the categorical feature. Every unique value in the category will be added as a feature.

# https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

batch_size=8
# small batch size leads to bigger BatchDataset
# shuffle: Whether to shuffle the data. Default: True. If set to False, sorts the data in alphanumeric order.
# seed: Optional random seed for shuffling and transformations.
seed=42
tr_data_dir_iterator=train_data_gen.flow_from_directory(
        directory="data/traindata/train/",
        batch_size=batch_size,
        class_mode='categorical',
        target_size=(224,224),
        seed=seed, 
        shuffle=True,
        subset='training')
valid_data_dir_iterator=train_data_gen.flow_from_directory(
        directory="data/traindata/train/",
        batch_size=batch_size,
        class_mode='categorical',
        target_size=(224,224),
        seed=seed, 
        shuffle=True,
        subset='validation'
)


test_data_dir_iterator=test_data_gen.flow_from_directory(
        directory="data/testdata/",
        batch_size=1,
        class_mode=None,
        color_mode="rgb",
        target_size=(224,224),
        shuffle=False,
        seed=seed
)
print(tr_data_dir_iterator,valid_data_dir_iterator,test_data_dir_iterator)




Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
<keras.preprocessing.image.DirectoryIterator object at 0x000001C28CCFB850> <keras.preprocessing.image.DirectoryIterator object at 0x000001C2D40F2280> <keras.preprocessing.image.DirectoryIterator object at 0x000001C2D40FCE80>


In [29]:
# vgg-16 network

model=Sequential()

model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same",activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1000,activation="relu"))
## as out puts can be of two types
model.add(Dense(units=2,activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_27 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_29 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 56, 56, 128)      0         
 g2D)                                                 

In [43]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# ModelCheckpoint helps us to save the model by monitoring a specific parameter of the model. In this case we are monitoring validation accuracy by passing val_acc to ModelCheckpoint. The model will only be saved to disk if the validation accuracy of the model in current epoch is greater than what it was in the last epoch.

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
# here patience is set to 20 means if accuracy is not increamenting after 20 epochs, exit from training.

hist = model.fit_generator(steps_per_epoch=tr_data_dir_iterator.n//tr_data_dir_iterator.batch_size,generator=tr_data_dir_iterator, validation_data= valid_data_dir_iterator, validation_steps=valid_data_dir_iterator.n//valid_data_dir_iterator.batch_size,epochs=40,callbacks=[checkpoint,early])
# The validation generator works exactly like the training generator. You define how many batches it will wield per epoch.

# The training generator will yield steps_per_epoch batches.
# When the epoch ends, the validation generator will yield validation_steps batches.
# steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
# validation_steps = TotalvalidationSamples / ValidationBatchSize
# https://stackoverflow.com/questions/45943675/meaning-of-validation-steps-in-keras-sequential-fit-generator-parameter-list